# Exploratory Data Analysis of Successful Film Trends
## Informing Decision-making for Microsoft's New Movie Studio

# 1. Business Understanding
Microsoft has identified an opportunity to enter the movie industry by establishing a new movie studio. However, as a technology company, they lack expertise in the domain of filmmaking. To make informed decisions about the type of films to produce, Microsoft needs to understand the current landscape of the movie industry.



The primary objective is to explore successful movies and translate the findings into actionable insights. Through exploratory data analysis, insights of the types of films doing well can be generated.These insights will serve as a foundation for determining the types of films the new studio should create to maximize their chances of achieving box office success.

This analysis aims to answer the following key questions:
* What are the top-performing genres in the movie industry?
* Are there any particular demographics or target audiences that have shown a strong preference for certain types of films?
* What is the average budget range for successful movies, and how does it correlate with their box office performance?
* Are there any seasonal or temporal factors influencing the success of films, such as release dates or holiday periods?
* Are there any specific marketing or promotional strategies employed by successful movies that contribute to their box office success?
* How do critical reviews and audience ratings impact the commercial success of movies?

# 2. Data Understanding
To gain a comprehensive understanding of the movie industry and address Microsoft's business problem, several types of data can be collected and analyzed. 
The datasets used for this anaysis contains information from movie sites. The files included  are `'tn.movie_budgets.csv'`, `'bom.movie_gross.csv'`, `'tmdb.movies.csv'` and a database called `'im.db'`.
Information contained in these files includes:
+ Genres
+ Movie Titles
+ Movie Ratings
+ Movie Budgets
+ Domestic and Foreign Gross Income
+ Release dates
+ Runtime Minutes
+ Popularity

## Relevance of this information
* We can be able to understand audience preferences by analyzing the performance of different genres which can help determine which genres are currently popular and have a higher likelihood of success.
* Analyzing ratings can help identify films that have been well-received and positively reviewed, suggesting higher chances of success.
* Analyzing the performance of films across different budget ranges can help identify patterns and determine optimal budget allocation strategies for Microsoft's new movie studio.
* Analyzing the performance of films in different markets can inform decisions regarding international distribution strategies and potential revenue streams.
* Analyzing runtime minutes can help identify patterns regarding film duration and potential audience preferences.
* Analyzing popularity metrics can help gauge the potential success and audience reception of movies.


# 3. Data Preparation

We first import the relevant packages for this analysis.

In [1]:
# importing the relevant packages
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt

#enabling plotting of visusalizations in the notebook
%matplotlib inline

## a) Previewing the Database Tables
The database `im.db` has some datasets that will be useful for this analysis.

In [4]:
#creating a database connection
conn = sqlite3.connect('im.db')

#querying the database to view the database tables
q1 = """SELECT name
        FROM sqlite_master
        WHERE type = 'table';
     """
#executing the query using pandas
pd.read_sql(q1,conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


From the result above, we will need only the `movie_basics` and `movie_ratings` table. 
## Loading the two tables
Starting with `movie_basics`

In [16]:
#previewing 'movie_basics' by selecting all columns from the table
q2 = """SELECT * 
        FROM movie_basics;
     """
#executing the query and assigning to variable name movie_basics
movie_basics = pd.read_sql(q2,conn)

#loading the first five rows
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


Now `movie_ratings`

In [9]:
#selecting all columns 
q3 = """SELECT *
        FROM movie_ratings;
     """
#executing the query and assigning to variable name movie_ratings
movie_ratings = pd.read_sql(q3,conn)
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


## Joining the two tables using movie_id to create one table
Both tables have _movie_id_, we join them to form one table with more information.

In [10]:
#query the database to join the tables
# we'll use a LEFT JOIN 
q4 = """SELECT * 
        FROM movie_basics
        LEFT JOIN movie_ratings USING(movie_id);        
     """
#executing the query and assigning to variable name movie_info
movie_info = pd.read_sql(q4,conn)
movie_info.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [27]:
#checking the info of this table
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
 6   averagerating    73856 non-null   float64
 7   numvotes         73856 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 8.9+ MB


## b) Loading the csv files
The csv files we'll use are `'tn.movie_budgets.csv'`, `'bom.movie_gross.csv'` and `'tmdb.movies.csv'`.

Previewing `'tn.movie_budgets.csv'` stored under data folder:

In [19]:
#using pandas to read the csv file and assigning it to variable name movie_budgets
movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')

#loading the first five rows
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [22]:
#checking the info about this file
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


Next `'bom.movie_gross.csv'`:

In [23]:
#reading and assigning to bom_movies
bom_movies = pd.read_csv('data/bom.movie_gross.csv')
bom_movies.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [25]:
#checking the info 
bom_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


Finally `'tmdb.movies.csv'`.

In [21]:
#reading and assigning to tmdb_movies
tmdb_movies = pd.read_csv('data/tmdb.movies.csv')
tmdb_movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [26]:
#checking the info
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB
